# Kings County Housing Prices Bakeoff

Below are a list of steps that you should take while trying to complete your bake-off entry.

## Step 1: Read in Data

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
import warnings
import folium
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import scipy.stats as stats
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [6]:
org_hous_df=pd.read_csv('Data/kc_house_data_train.csv', index_col='id').drop('Unnamed: 0', axis=1)
org_hous_df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
2591820310,20141006T000000,365000.00,4,2.25,2070,8893,2.00,0,0,4,8,2070,0,1986,0,98058,47.44,-122.16,2390,7700
7974200820,20140821T000000,865000.00,5,3.00,2900,6730,1.00,0,0,5,8,1830,1070,1977,0,98115,47.68,-122.28,2370,6283
7701450110,20140815T000000,1038000.00,4,2.50,3770,10893,2.00,0,2,3,11,3770,0,1997,0,98006,47.56,-122.13,3710,9685
9522300010,20150331T000000,1490000.00,3,3.50,4560,14608,2.00,0,2,3,12,4560,0,1990,0,98034,47.70,-122.23,4050,14226
9510861140,20140714T000000,711000.00,3,2.50,2550,5376,2.00,0,0,3,9,2550,0,2004,0,98052,47.66,-122.08,2250,4050


In [7]:
top5codes_price = org_hous_df.groupby("zipcode")["price"].mean().sort_values(ascending = False)[:5]
mean_price = org_hous_df.price.mean()
print(top5codes_price)

zipcode
98039   2202790.00
98004   1396882.89
98040   1183018.00
98112   1119457.66
98102    933671.47
Name: price, dtype: float64


In [8]:
count_by_zip = org_hous_df.groupby("zipcode")["price"].count().sort_values(ascending = False)[:70]
count_by_zip

zipcode
98052    474
98115    465
98103    461
98038    455
98117    437
        ... 
98102     80
98010     74
98024     65
98148     43
98039     42
Name: price, Length: 70, dtype: int64

In [9]:
mean_price_by_zip = org_hous_df.groupby("zipcode")["price"].mean().sort_values(ascending = False)[:70]
mean_price_by_zip = mean_price_by_zip.to_frame()
mean_price_by_zip['count'] = count_by_zip
mean_price_by_zip

,price,count
zipcode,,
98039,2202790.00,42
98004,1396882.89,237
98040,1183018.00,229
98112,1119457.66,216
98102,933671.47,80
...,...,...
98001,281998.76,284
98148,272082.33,43
98032,251602.23,103


In [31]:
no_bedrooms = org_hous_df[org_hous_df['bedrooms'] == 0]['price']
no_bathrooms = org_hous_df[org_hous_df['bathrooms'] == 0]['price']
bathroom_pvalue = stats.ttest_ind(no_bedrooms, no_bathrooms)[1]
bathroom_pvalue

0.6824544554785027

In [34]:
alpha=.05

def stattest_hypo(pvalue):
    if pvalue < alpha:
        print("The "+ pvalue + " is less than alpha, reject null-hypothesis")
    else:
        print("The "+ pvalue + " is more than alpha, accept null-hypothesis")
        
stattest_hypo(bathroom_pvalue)

TypeError: can only concatenate str (not "numpy.float64") to str

In [11]:
alpha = 0.05
multi_p_val = stats.ttest_ind(multi.price, single.price, equal_var=False)[1]
print("Multiple Floor vs. Single Floor T-test P Value: ", multi_p_val)
if multi_p_val < alpha:
    print("The P value is less than alpha, reject null-hypothesis")

NameError: name 'multi' is not defined

## Step 2: Exploratory Data Analysis 
    
Become familiar with the data.  Look to see if there are any extreme values.  

Additionally create data visualizations to determine if there are any relationships between your features and your target variables.  

In [ ]:
org_hous_df.describe()

In [ ]:
#Identify colinearity for all features with price with plots
fig, axes = plt.subplots(6, 3, figsize=(20,30), sharey=True)
for ax, column in zip(axes.flatten(), org_hous_df.columns[2:]):
    ax.scatter(org_hous_df[column], org_hous_df['price'], label=column, alpha=.05)
    ax.set_title(f'Price vs {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Price')

In [ ]:
org_hous_df.groupby("view")["price"].mean().plot(kind='bar', figsize=(10,10))

In [ ]:
corr = org_hous_df.corr().abs()
sns.set(rc={'figure.figsize':(12,9)})
sns.heatmap(corr[:20], xticklabels=corr[:20].columns, \
            yticklabels=corr[:20].columns, \
            annot=False, cmap='Blues_r', center= 0)
plt.title('Degrees of Correlation in dataset');

In [ ]:
sns.set_palette('Blues_r')
x = ['98039', '98004', '98040', '98112', '98102']
y = [2.202790e+06, 1.396883e+06, 1.183018e+06, 1.119458e+06, 9.336715e+05]
fig, ax = plt.subplots(figsize = (15, 5))
ax = sns.barplot(x=x,y=y, data=org_hous_df)
ax.ticklabel_format(style='scientific', axis='y')

ax.set(xlabel='Zip Code', ylabel='Avg. Price')
plt.title("Average Home Price")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (12,6))
sns.distplot(org_hous_df['price'], bins=50, color = 'b')
ax.set(xlim=[0, 5000000], xlabel='Sale Price', ylabel='Sale Count',
       title='Sale Distribution')

In [ ]:
sns.set_theme(style='ticks', palette="rocket")
org_hous_df.hist(bins=50, figsize=(20,20))
plt.show()

In [ ]:
var = 'waterfront'
data = pd.concat([org_hous_df['price'], org_hous_df[var]], axis=1)
fig, ax = plt.subplots(figsize=(10,8))
fig = sns.boxplot(x=var, y='price', data=data)

In [ ]:
var = 'grade'
data = pd.concat([org_hous_df['grade'], org_hous_df[var]], axis=1)
fig, ax = plt.subplots(figsize=(10,8))
fig = sns.boxplot(x=var, y='price', data=data)

In [ ]:
sns.scatterplot(data = org_hous_df, x = org_hous_df['price'], y = org_hous_df['sqft_living'])

plt.show()

In [ ]:
#folium Map using coordinates
kc_map = folium.Map(location = [47.5480, -121.9836], tiles = 'OpenStreetMap', zoom_start = 9)

kc_coord = list(zip(org_hous_df['lat'], org_hous_df['long']))

for coord in kc_coord:
    folium.Marker(location = coord).add_to(kc_map)

#kc_map

## Step 3: Clean up any issues (extreme values, etc.) with the data.  

Remember that you can't just delete rows with extreme values. Similar observations might be present in the holdout data set, and you can't just delete those rows and not have a prediction for it. 

In [ ]:
bedrooms - 33 bedroom house on Google Maps seems to be typo for a 3 bedroom house, twelve "studios" with no bedrooms
bathrooms - eight with no bathrooms

In [ ]:
#correct 33 bedroom home
org_hous_df['bedrooms'].replace(33, 3, inplace=True)

In [ ]:
#save cleaned df
org_hous_df.to_csv('~/Data', index=False)

## Step 4: Generate new features that you think could be important.

After doing this, you will want to go back to steps 2 and 3 to investigate these new features.

In [ ]:
#yard_space column
org_hous_df['yard_space'] = org_hous_df.sqft_lot - (org_hous_df.sqft_living / org_hous_df.floors)

In [ ]:
#multi_floor_column
org_hous_df['multi_floor'] = [0 if x == 1 else 1 for x in org_hous_df.floors] 

In [ ]:
#Basement coulumn
org_hous_df['has_basement'] = [0 if x == 0 else 1 for x in org_hous_df.sqft_basement]

In [ ]:
#sale_month column to explore affeect of seasons
org_hous_df['sale_month'] = pd.DatetimeIndex(org_hous_df['date']).month

In [ ]:
#Year Renovated coulumn
org_hous_df['yr_built_reno'] = np.where((org_hous_df['yr_built'] < 1974) & (org_hous_df['yr_renovated'] == 0), 0,1)

### 4.1) Identify a categorical variable in the data set and create dummy columns.

In [ ]:
# identified zipcode,grade and view
#org_hous_df=pd.get_dummies(org_hous_df, columns=['zipcode'], drop_first=True)
#org_hous_df=pd.get_dummies(org_hous_df, columns=['grade'], drop_first=True)
#org_hous_df=pd.get_dummies(org_hous_df, columns=['view'], drop_first=True)
org_hous_df.shape

### 4.3) There are columns for when the house was built and when it was renovated.  How could you use these columns to create a new column?

In [ ]:
ols(formula='price~sqft_living', data= org_hous_df).fit().summary()

### <ins>Non-linear transformations</ins>

### 4.4) Create a polynomial feature for two of your continuous variables.

In [ ]:
#org_hous_df['sqft_living^3'] = org_hous_df['sqft_living']**3
#org_hous_df['sqft_living^2'] = org_hous_df['sqft_living']**2
#org_hous_df['sqft_basement^2'] = org_hous_df['sqft_basement']**2
org_hous_df

### 4.5) Create an interaction feature between a binary variable (dummy variable) and a continuous variable.

In [ ]:
#sqft of homes sold with the best view
#org_hous_df['bestview_sqftliving'] = org_hous_df['view_4']*org_hous_df['sqft_living']

In [ ]:
#zipcode with 474 houses sqft
#org_hous_df['biggestzipcode_sqftliving'] = org_hous_df['zipcode_98052']*org_hous_df['sqft_living']

In [ ]:
continuous = ['sqft_living','sqft_lot','sqft_above','sqft_basement','yr_built','sqft_living15','sqft_lot15']

for column in continuous:
    sns.jointplot(x=column, y="price", data=org_hous_df, kind='reg', label=column,joint_kws={'line_kws':{'color':'red'}})
    plt.legend()
    plt.show()

### 4.6) SKLearn Identifying Features 

In [ ]:
features = ['second_sale', 'zipcode_98004', 'zipcode_98038', 'zipcode_98116','zipcode_98117','zipcode_98052',/
            'sale_month', 'yard_space', 'grade_3','grade_4','grade_5','grade_6','grade_7','grade_8','grade_9',/
            'grade_10','grade_11','grade_12','grade_13','view_1','view_2','view_3', 'view_4', 'sqft_living^2',/
            'been_renovated', 'sqft_living^3','sqft_basement^2', 'bestview_sqftliving', 'biggestzipcode_sqftliving']

In [ ]:
df_features = org_hous_df.filter(org_hous_df.columns, axis=1)
df_features.drop(columns=['lat','long','date', 'id','price'], inplace=True)

In [ ]:
df_features.drop( 'long', axis=1, inplace=True)

In [ ]:
df_features.drop( 'date', axis=1, inplace=True)

In [ ]:
df_features.drop( 'id', axis=1, inplace=True)

In [ ]:
df_features.drop( 'price', axis=1, inplace=True)

In [ ]:
target = org_hous_df['price']

In [ ]:
len(df_features.columns)

In [ ]:
from sklearn.linear_model import LinearRegression

#instantiate a linear regression object
lm = LinearRegression()

#fit the linear regression to the data
lm = lm.fit(df_features, target)

#access output
print(lm.intercept_)
print(lm.coef_)
print("R^2: ", lm.score(df_features, target))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
poly_data = poly.fit_transform(df_features)

In [ ]:
df_features.isnull().value_counts

## Step 5: Model Evaluation



## 5.1)  Fit an initial model and check the errors to see if the model assumptions are being met. If need be, go back to steps 3 and 4 to improve your model. 

In [ ]:
ols(formula='price~C(zipcode)+sqft_living+view', data= df_features).fit().summary()

In [ ]:
len(df_features)

In [ ]:
poly_columns = poly.get_feature_names(df_features.columns)

In [ ]:
len(poly_columns)

In [ ]:
df_features

In [ ]:
df_poly = pd.DataFrame(poly_data, columns=poly_columns)

In [ ]:
df_poly.head()

In [ ]:
poly_3 = PolynomialFeatures(degree=3, include_bias=False)
poly3_data = poly_3.fit_transform(df_features)
poly3_columns = poly_3.get_feature_names(df_features.columns)
df_poly3 = pd.DataFrame(poly3_data, columns=poly3_columns)

In [ ]:
#instantiate a linear regression object
lm_2 = LinearRegression()

#fit the linear regression to the data
lm_2 = lm_2.fit(df_poly, target)

#access output
print(lm_2.intercept_)
print(lm_2.coef_)
print("R^2: ", lm_2.score(df_poly, target))

In [ ]:
#instantiate a linear regression object
lm_3 = LinearRegression()

#fit the linear regression to the data
lm_3 = lm_3.fit(df_poly3, target)

#access output
# print(lm_3.intercept_)
# print(lm_3.coef_)
print("R^2: ", lm_3.score(df_poly3, target))

In [ ]:
df_features.columns

### 5.2) Perform a train-test split of the data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_features, target, random_state=9,test_size=0.3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_features, target, test_size=0.3, random_state=34)
print(len(X_train), len(X_test), len(y_train), len(y_test))

### 5.3) Fit the model to the training data.

In [ ]:
y_train

In [ ]:
target.head()

In [ ]:
scaler = StandardScaler()

# fit the scaler to the training data
scaler.fit(X_train)

#transform the training data
scaled_data = scaler.transform(X_train)

#create dataframe
X_train = pd.DataFrame(data=scaled_data, columns=df_features.columns)

#transform the testing dat
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=df_features.columns)

### 5.4) Use the model to predict on the training set and the test set.

In [ ]:
#check the shape of the results
print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Training set - Features: ", X_test.shape, "Target: ",y_test.shape)

### 5.5) Evaluate the training and test predictions using RMSE.

### 5.6) Determine if your model is overfit.

In [ ]:
def test_data(df_features, target):
    testtrainsplit
    lm = LinearRegression()
    lm.fit()
    return RMSE

In [ ]:
df1_rmse = test_data(org_hous_df)

## Step 6: Utilize some different feature selection techniques before or in conjuction with fitting your models. You will have to repeat steps 5.3 through 5.6 to determine how your new model is performing. 

### 6.1) Utilize a filter method to identify some features to remove from the model.  

In [ ]:
X_train.shape

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

selector = SelectKBest(f_regression, k='all')

selector.fit(X_train, y_train)

In [ ]:
selector.get_support()

In [ ]:
selected_columns = X_train.columns[selector.get_support()]
removed_columns = X_train.columns[~selector.get_support()]
X_train = X_train[selected_columns]
X_test = X_test[selected_columns]

In [ ]:
list(selected_columns)

### 6.2) After removing the features, re-run Step 5 and see if your new model performs better than the old model.

In [ ]:

sns.set(style="white")


# Compute the correlation matrix
corr = X_train.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df_features.shape

In [ ]:
target.shape

In [ ]:
#instantiate a linear regression object
lm_kbest = LinearRegression()

#fit the linear regression to the data
lm_kbest.fit(df_features, target)

y_train_kbest = lm_kbest.predict(df_features)


trainK_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbest))


print('Training Root Mean Squared Error:' , trainK_rmse)

y_kbest = lm_kbest.predict(X_test)

testK_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbest))

print('Testing Root Mean Squared Error:' , testK_rmse)

In [ ]:
import pickle

pickle_out = open("model.pickle","wb")

pickle.dump(lm, pickle_out)

pickle_out.close()

In [ ]:
pickle_out = open("column_selection.pickle","wb")

pickle.dump(df_features.columns, pickle_out)

pickle_out.close()

## Step 7: Compare the RMSE of your different models that use different features and determine the best model overall.

## Step 8:  Refit your best model to the entire dataset.

## Step 9: Save your final model using pickle.

https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/